In [1]:
import torch
import torch.nn as nn
from ultralytics import YOLO
import cv2
import os
import time
from pathlib import Path

try:
    _original_torch_load = torch.load
    def _patched_torch_load(f, *args, **kwargs):
        kwargs['weights_only'] = False
        return _original_torch_load(f, *args, **kwargs)
    torch.load = _patched_torch_load
except Exception:
    pass

# Check for hardware acceleration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [2]:
BACKEND_DIR = Path("../backend")
MODEL_N = BACKEND_DIR / "yolov8n.pt"

# Force use Nano for better performance on local machines
if MODEL_N.exists():
    model_path = str(MODEL_N)
else:
    model_path = "yolov8n.pt" # will download if missing

print(f"Loading Nano model for performance: {model_path}")
model = YOLO(model_path).to(device)

Loading Nano model for performance: ..\backend\yolov8n.pt


In [3]:
cap = cv2.VideoCapture(0)
TARGET_CLASSES = ["person", "cell phone", "laptop", "book"]

frame_count = 0
skip_frames = 3 # Only run AI every 3 frames
last_results = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    frame_count += 1
    h, w = frame.shape[:2]

    # Run detection only on specific frames
    if frame_count % skip_frames == 0:
        # Downscale frame for faster processing (optional)
        results = model(frame, conf=0.4, imgsz=320, verbose=False)
        last_results = results

    # Draw last known results on every frame to keep visual smooth
    if last_results:
        for r in last_results:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                label = r.names[cls_id]
                if label in TARGET_CLASSES:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display FPS
    cv2.putText(frame, f"FPS Mode: Skip {skip_frames}", (10, h - 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("YOLOv8 Optimized Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27: break

cap.release()
cv2.destroyAllWindows()